In [1]:
import operator as op
import src.ea.ea_lib as ea
import src.ea.problem_data as pb
import src.ea.utilities as ut
import random as rd
import time
import numpy as np

In [2]:
#parameters
rd.seed(1)
population_size = 200
tournament_size = 4
problem_name = "spect"
levels_back = 2
n_rows = 4
n_columns = 9
allow_input_to_output = False
inputs_available_to_all_columns = False
functions = [op.add,op.sub,op.mul,ut.safe_divide_one]
objectives = [
    ea.Objective(name="acc0", to_max = True, best=1, worst=0),
    ea.Objective(name="acc1", to_max = True, best=1, worst=0)
    ]

In [3]:
#Definition
dataset = pb.Dataset()
dataset.load_problem(name = problem_name)
cgp = ea.CGP_Representation(
            dataset.x_train.shape[1]
            ,1
            ,levels_back
            ,n_rows 
            ,n_columns
            ,allow_input_to_output
            ,inputs_available_to_all_columns
            ,*functions)

In [4]:
#Execution
start_t = time.time()

generation = 0
#Random initial population. Specific initial conditions for the population can be specified here
graphs = [cgp.create_random(seed = rd.random()) for _ in range(population_size)]
#create instances of Individual to be grouped in the population
population = [ea.Individual(r,generation) for r in graphs]
for ind in population:
    outputs = {}
    for i,data_row in enumerate(dataset.x_train):
        output_dict = ind.representation.evaluate(data_row = data_row)
        #Extracting the 0th index output gene
        output = output_dict[0]
        #The raw output needs to be transformed, in this case it is simply rounded to the closest int
        output_rounded = ut.custom_round(output)
        outputs[i] = output_rounded
    #Each objective has its own evaluation method
    acc0 = ut.accuracy_in_label(y=dataset.y_train, y_output=outputs, label = 0)
    acc1 = ut.accuracy_in_label(y=dataset.y_train, y_output=outputs, label = 1)
    ind.update_evaluation(objective=objectives[0],value=acc0)
    ind.update_evaluation(objective=objectives[1],value=acc1)
    ind.update_semantics_all(semantics_all = outputs)

print("Execution time: ",str(time.time()-start_t))

evals0 = []
evals1 = []
actives = []
for i,ind in enumerate(population):
    evals0.append(ind.evaluations[objectives[0].name])
    evals1.append(ind.evaluations[objectives[1].name])
    actives.append(len(ind.representation.active_genotype))
print("Mean acc0", np.mean(evals0))
print("Mean acc1", np.mean(evals1))
print("Mean active nodes", np.mean(actives))

Execution time:  2.2679591178894043
Mean acc0 0.2572659176029963
Mean acc1 0.1350561797752809
Mean active nodes 7.9


In [5]:
nsgaii_objectives = ea.get_nsgaii_objectives()
sorted_nsga2 = ea.fast_nondominated_sort(population=population,conflicting_objectives=objectives, nsgaii_objectives=nsgaii_objectives)
for ind in sorted_nsga2:
    print(ind.evaluations)

{'acc0': 0.5880149812734082, 'acc1': 0.0, 'front': 1, 'cd': inf}
{'acc0': 0.0, 'acc1': 0.41198501872659177, 'front': 1, 'cd': inf}
{'acc0': 0.5880149812734082, 'acc1': 0.0, 'front': 1, 'cd': inf}
{'acc0': 0.0, 'acc1': 0.41198501872659177, 'front': 1, 'cd': inf}
{'acc0': 0.3782771535580524, 'acc1': 0.2696629213483146, 'front': 1, 'cd': 0.5647944412275623}
{'acc0': 0.250936329588015, 'acc1': 0.2958801498127341, 'front': 1, 'cd': 0.41302837290098443}
{'acc0': 0.41198501872659177, 'acc1': 0.1760299625468165, 'front': 1, 'cd': 0.37649102489866826}
{'acc0': 0.18352059925093633, 'acc1': 0.30337078651685395, 'front': 1, 'cd': 0.33202084539664156}
{'acc0': 0.50187265917603, 'acc1': 0.1198501872659176, 'front': 1, 'cd': 0.31557614360162123}
{'acc0': 0.5468164794007491, 'acc1': 0.06367041198501873, 'front': 1, 'cd': 0.30370584829183556}
{'acc0': 0.11610486891385768, 'acc1': 0.36329588014981273, 'front': 1, 'cd': 0.29108280254777064}
{'acc0': 0.4606741573033708, 'acc1': 0.17228464419475656, 'front

In [12]:
graph = population[0].representation

#point mutation
nodes_list = list(graph.genotype.values())
n_function_genes = graph.max_lenght + sum([n.function_arity for n in graph.genotype.values()])
n_genes = n_function_genes + graph.n_outputs
mutations = int(n_genes * percentage / 100)
    for _ in range(mutations):
        int_to_mutate = rd.randint(0, n_genes-1)
        mutate_output_index = int_to_mutate + 1 - n_function_genes
        if mutate_output_index >= 0:
            print("mutate output")
            #self.mutate_output_gene(graph.output_gene[mutate_output_index])
        else:
            node = rd.choice(nodes_list)
            int_mutation = rd.randint(0, node.function_arity)
            if int_mutation == node.function_arity:
                print("mutate_function")
            else:
                print("mutate input", int_mutation)

36


108